In [ ]:
def sensitivity_analysis(func_id, X_sample, y_sample, n_points=50):
    """
    Perform sensitivity analysis by varying each dimension independently
    
    Args:
        func_id: Function ID
        X_sample: Observed inputs
        y_sample: Observed outputs
        n_points: Number of points to sample along each dimension
    """
    from utils.bayesian_optimization import fit_gp
    
    dim = X_sample.shape[1]
    
    # Fit GP
    gp = fit_gp(X_sample, y_sample)
    
    # Use best point as baseline
    best_idx = np.argmax(y_sample)
    baseline = X_sample[best_idx].copy()
    
    # Create figure
    n_cols = min(4, dim)
    n_rows = (dim + n_cols - 1) // n_cols
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(5*n_cols, 4*n_rows))
    if dim == 1:
        axes = np.array([axes])
    axes = axes.flatten()
    
    print(f"\n{'='*70}")
    print(f"Function {func_id} ({dim}D) - Sensitivity Analysis")
    print(f"{'='*70}")
    print(f"Baseline (best point): {baseline}")
    print(f"Baseline output: {y_sample[best_idx]:.6f}\n")
    
    # Analyze each dimension
    for d in range(dim):
        # Vary dimension d from 0 to 1
        test_points = np.tile(baseline, (n_points, 1))
        test_points[:, d] = np.linspace(0, 1, n_points)
        
        # Get GP predictions
        mu, sigma = gp.predict(test_points, return_std=True)
        
        # Plot
        ax = axes[d]
        ax.plot(test_points[:, d], mu, 'b-', linewidth=2, label='GP mean')
        ax.fill_between(test_points[:, d], mu - sigma, mu + sigma, alpha=0.3, label='±1σ')
        ax.axvline(baseline[d], color='r', linestyle='--', linewidth=2, label=f'Current: {baseline[d]:.3f}')
        ax.axhline(y_sample[best_idx], color='g', linestyle=':', linewidth=1, alpha=0.5)
        
        # Mark observed points in this dimension
        obs_in_dim = X_sample[:, d]
        ax.scatter(obs_in_dim, y_sample, c='orange', s=50, alpha=0.6, zorder=5)
        
        ax.set_xlabel(f'Dimension {d+1}', fontsize=11)
        ax.set_ylabel('Predicted Output', fontsize=11)
        ax.set_title(f'Dim {d+1} Sensitivity', fontsize=12, fontweight='bold')
        ax.legend(fontsize=9)
        ax.grid(alpha=0.3)
        
        # Calculate gradient (sensitivity)
        gradient = np.gradient(mu)
        max_gradient = np.max(np.abs(gradient))
        print(f"Dimension {d+1}: Max gradient = {max_gradient:.4f} {'(steep - important)' if max_gradient > 1 else '(flat - less important)'}")
    
    # Hide unused subplots
    for idx in range(dim, len(axes)):
        axes[idx].set_visible(False)
    
    plt.tight_layout()
    plt.show()
    print(f"{'='*70}\n")


# Run sensitivity analysis for all functions
print("\n" + "="*70)
print("SENSITIVITY ANALYSIS - Understanding Feature Effects")
print("="*70)
print("Shows how each dimension affects output when varied independently")
print("Steep slopes = important features, Flat = less important\n")

for func_id in range(1, 9):
    X = inputs[func_id]
    y = outputs[func_id]
    sensitivity_analysis(func_id, X, y, n_points=50)

# Week 3 - Bayesian Optimization

Generate optimized recommendations for Week 3 using utility modules.

## Setup

In [ ]:
import numpy as np
import sys
sys.path.append('..')  # Add parent directory to path

# Import utility modules
from utils.bayesian_optimization import propose_next_point, get_strategy
from utils.data_utils import (
    load_week_data,
    save_week_data,
    combine_with_week_results, 
    print_data_summary
)
from utils.visualization import visualize_all_functions

## 1. Load Week 2 Data

Load the combined data from Week 2 (includes initial + Week 1 results)

In [ ]:
# Load Week 2 clean data (initial + Week 1)
inputs, outputs = load_week_data("../week 2/week2_clean_data.npz")
print_data_summary(inputs, outputs, "Week 2 Data (Initial + Week 1)")

## 2. Add Week 2 Results

**TODO: Update with actual Week 2 results when you receive them**

In [ ]:
# Week 2 submitted points (from Week 2 recommendations)
week2_inputs = {
    1: np.array([0.400174, 0.668743]),
    2: np.array([0.700073, 0.125362]),
    3: np.array([0.994082, 1.000000, 0.405235]),
    4: np.array([0.431409, 0.391102, 0.432466, 0.377774]),
    5: np.array([0.381844, 0.303415, 1.000000, 1.000000]),
    6: np.array([0.755469, 0.270580, 0.644099, 0.672228, 0.162862]),
    7: np.array([0.000000, 0.342263, 0.707844, 0.246481, 0.405689, 0.778028]),
    8: np.array([0.044443, 0.140302, 0.000000, 0.042659, 1.000000, 0.088025, 0.000000, 0.953632])
}

# Week 2 outputs (received from black box)
week2_outputs = {
    1: 3.1126972846093504e-39,
    2: 0.6138474827757134,
    3: -0.04985944307471363,
    4: 0.3523251399573444,
    5: 1688.0687842580955,
    6: -0.5209883632002658,
    7: 1.7844832290542527,
    8: 9.6492817382496
}

# Combine with Week 2
inputs, outputs = combine_with_week_results(inputs, outputs, week2_inputs, week2_outputs)
print_data_summary(inputs, outputs, "After Week 2")

## 3. Save Week 3 Clean Data

In [ ]:
# Save combined data for Week 4
save_week_data(inputs, outputs, "week3_clean_data.npz")

## 4. Generate Week 3 Recommendations

In [ ]:
print("=" * 70)
print("WEEK 3 BAYESIAN OPTIMIZATION RECOMMENDATIONS")
print("ADJUSTED BASED ON SENSITIVITY ANALYSIS")
print("=" * 70)

week3_recommendations = {}

for func_id in range(1, 9):
    X = inputs[func_id]
    y = outputs[func_id]
    dim = X.shape[1]
    
    # Bounds: all inputs in [0, 1]
    bounds = np.array([[0, 1]] * dim)
    
    # ADJUSTED STRATEGIES based on sensitivity analysis
    if func_id == 1:
        # Function 1: Lost narrow peak, flat gradients - HIGH EXPLORATION
        acq = 'EI'
        xi = 0.1  # INCREASED from 0.001
        kappa = 1.0
        reason = "High exploration - relocate lost narrow peak (flat gradients)"
    elif func_id == 2:
        # Function 2: Slight improvement, continue moderate exploration
        acq = 'EI'
        xi = 0.01
        kappa = 1.5
        reason = "Moderate exploration - working (flat but improving)"
    elif func_id == 3:
        # Function 3: Stuck on plateau, flat gradients - INCREASED EXPLORATION
        acq = 'EI'
        xi = 0.05  # INCREASED from 0.01
        kappa = 1.5
        reason = "Increased exploration - escape plateau (all dims flat)"
    elif func_id == 4:
        # Function 4: Breakthrough! Keep strategy
        acq = 'EI'
        xi = 0.01
        kappa = 1.5
        reason = "Keep strategy - breakthrough working (balanced gradients)"
    elif func_id == 5:
        # Function 5: Dims 3&4 at max, continue exploitation
        acq = 'EI'
        xi = 0.001
        kappa = 1.0
        reason = "Exploitation - dims 3&4 maxed (steep gradients)"
    elif func_id == 6:
        # Function 6: Near plateau, fine-tune
        acq = 'EI'
        xi = 0.001
        kappa = 1.0
        reason = "Exploitation - near plateau (flat gradients)"
    elif func_id in [7, 8]:
        # Functions 7-8: Near convergence, fine-tune
        acq = 'UCB'
        xi = 0.0001
        kappa = 0.5
        reason = "Fine-tuning - near convergence (all flat)"
    
    # Get recommendation
    next_point, gp = propose_next_point(
        X, y, bounds,
        acq_func=acq,
        xi=xi,
        kappa=kappa,
        n_restarts=50
    )
    
    week3_recommendations[func_id] = next_point
    
    # Display results
    current_best_idx = np.argmax(y)
    current_best = y[current_best_idx]
    
    print(f"\n{'='*70}")
    print(f"FUNCTION {func_id} ({dim}D)")
    print(f"{'='*70}")
    print(f"Data points: {len(y)}")
    print(f"Current best: {current_best:.6f}")
    print(f"Strategy: {acq} (xi={xi}, kappa={kappa})")
    print(f"Reason: {reason}")
    print(f"Week 3 recommended point: {next_point}")
    pred_mean = gp.predict(next_point.reshape(1, -1))[0]
    pred_std = gp.predict(next_point.reshape(1, -1), return_std=True)[1][0]
    print(f"Predicted: {pred_mean:.6f} ± {pred_std:.6f}")

print(f"\n{'='*70}")
print("SUMMARY - Week 3 Points to Submit:")
print(f"{'='*70}")
for func_id in range(1, 9):
    point = week3_recommendations[func_id]
    print(f"Function {func_id}: {' – '.join([f'{x:.2f}' for x in point])}")

## 5. Visualizations

In [ ]:
# Generate all visualizations
visualize_all_functions(inputs, outputs, week3_recommendations)

## 6. Sensitivity Analysis

Analyze individual feature effects by varying one dimension at a time